In [1]:
import torch
import torch.nn as nn

In [171]:
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()  # Calls the constructor of the parent class `nn.Module`.
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out


In [172]:
#create dataset 
features = torch.rand(10,5)

model = Model(features.shape[1])

model(features)

tensor([[0.5265],
        [0.5056],
        [0.4430],
        [0.5762],
        [0.4943],
        [0.5010],
        [0.5955],
        [0.5871],
        [0.5436],
        [0.4817]], grad_fn=<SigmoidBackward0>)

In [173]:
model.linear.weight

Parameter containing:
tensor([[-0.2985,  0.3025, -0.0298,  0.3605, -0.0945]], requires_grad=True)

In [174]:
model.linear.bias

Parameter containing:
tensor([-0.0698], requires_grad=True)

In [175]:
!pip install torchinfo

In [176]:
from torchinfo import summary
summary(model, input_size=(10, 5), col_names=["input_size", "output_size", "num_params"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Model                                    [10, 5]                   [10, 1]                   --
├─Linear: 1-1                            [10, 5]                   [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

## Lets make more complex layer

In [11]:
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()  # Calls the constructor of the parent class `nn.Module`.
        self.linear1 = nn.Linear(num_features, 3)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear1(features)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out
#we can use sequential container instead of this hectic forward pass

In [7]:
features = torch.rand(10,5)
model = Model(features.shape[1])
model(features)

tensor([[0.4523],
        [0.4489],
        [0.4518],
        [0.4443],
        [0.4502],
        [0.4518],
        [0.4497],
        [0.4501],
        [0.4463],
        [0.4572]], grad_fn=<SigmoidBackward0>)

In [8]:
model.linear1.weight

Parameter containing:
tensor([[ 0.1205,  0.2673, -0.2586,  0.1084,  0.2983],
        [-0.4218, -0.2715, -0.3965,  0.0228, -0.1776],
        [-0.0806,  0.0606,  0.1598,  0.2887,  0.0290]], requires_grad=True)

In [9]:
model.linear2.weight

Parameter containing:
tensor([[-0.2836,  0.1986,  0.1667]], requires_grad=True)

# Let's try building pipeline

In [12]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [14]:
df.shape

(569, 33)

In [24]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [26]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [28]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

### Define a model

In [29]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [31]:
loss_function = nn.BCELoss()

In [32]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7718549966812134
Epoch: 2, Loss: 0.7476724982261658
Epoch: 3, Loss: 0.7249446511268616
Epoch: 4, Loss: 0.7035971879959106
Epoch: 5, Loss: 0.6835529804229736
Epoch: 6, Loss: 0.6647334694862366
Epoch: 7, Loss: 0.6470600962638855
Epoch: 8, Loss: 0.6304558515548706
Epoch: 9, Loss: 0.6148462295532227
Epoch: 10, Loss: 0.600159764289856
Epoch: 11, Loss: 0.5863288640975952
Epoch: 12, Loss: 0.5732900500297546
Epoch: 13, Loss: 0.5609841346740723
Epoch: 14, Loss: 0.5493561625480652
Epoch: 15, Loss: 0.5383554100990295
Epoch: 16, Loss: 0.527935266494751
Epoch: 17, Loss: 0.5180526375770569
Epoch: 18, Loss: 0.5086682438850403
Epoch: 19, Loss: 0.49974602460861206
Epoch: 20, Loss: 0.49125292897224426
Epoch: 21, Loss: 0.4831588566303253
Epoch: 22, Loss: 0.4754360318183899
Epoch: 23, Loss: 0.46805912256240845
Epoch: 24, Loss: 0.4610050320625305
Epoch: 25, Loss: 0.4542524814605713
Epoch: 26, Loss: 0.4477817416191101
Epoch: 27, Loss: 0.44157513976097107
Epoch: 28, Loss: 0.435616135597229


In [33]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.5277008414268494
